In [1]:
import json
import time
import subprocess
import math
import regex
import sys
import os

import spacy
from tqdm import tqdm
from nltk.corpus import wordnet as wn
import numpy as np
from sklearn.model_selection import KFold

from json_parser import Parser
from common import BBT_PEOPLE, BoundingBox
from image_processing import ObjectDetector
import language_processing as lp
import utils
from utils import split_data_set
import inference as inf

In [2]:
with open('hold_base.lp') as f:
    background_knowledge = f.read().splitlines()

parser = Parser()

# With Object Detections but all questions

In [3]:
def one_iter_gt(train_set, test_set):
    # 3. Test on test set
    jacc_score = 0
    num_questions = 0
    parsing_error = []
    not_full_score = []
    zero_score = []

    for test in test_set:
        pred_ans_idx = inf.inference(test)
        if pred_ans_idx == [-1]:
            parsing_error.append(test['qid'])
            continue

        score = inf.get_jaccard_score(pred_ans_idx, test)

        if 0 < score < 1:
            not_full_score.append(test['qid'])
        elif score == 0:
            zero_score.append(test['qid'])

        jacc_score += score
        num_questions += 1

    ##########################################################
    
    # 4. Print out result
    print(F'# tests:     {num_questions}')
    print(F'jacc score:   {jacc_score}')
    print(F'norm jacc score: {jacc_score / num_questions}')
    print()

    print('PARSING ERROR')
    print(parsing_error)
    print()

    print('NOT FULL SCORE')
    print(not_full_score)
    print()

    print('ZERO SCORE')
    print(zero_score)
    
    return jacc_score, num_questions

In [4]:
with open('hold_questions.json') as f:
    attempted_question = json.load(f)

print(F'# total tests: {len(attempted_question)}')

a_q_dict = dict()

for q in attempted_question:
    qid = q['qid']
    a_q_dict[qid] = q
    
kf = KFold(n_splits=5)
kf.get_n_splits(attempted_question)

splits = kf.split(attempted_question)

run_id = 0
total_jacc_score = 0
total_questions = 0

# total tests: 886


# Run 1

In [5]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_od_all', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1675


In [6]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     177
jacc score:   25.833333333333332
norm jacc score: 0.14595103578154425

PARSING ERROR
[91833]

NOT FULL SCORE
[87060, 71186, 112266, 33374, 51101, 61212]

ZERO SCORE
[59566, 45810, 113521, 20399, 12972, 107346, 88056, 102118, 105776, 85355, 69808, 117818, 86427, 98156, 1260, 18350, 85689, 65891, 11924, 88747, 61756, 75239, 72563, 76199, 95090, 62856, 115774, 9143, 77658, 22847, 52058, 67221, 89923, 25720, 60940, 33519, 66534, 59096, 63936, 11794, 83600, 73445, 50348, 65848, 38557, 55976, 36135, 84235, 90819, 32735, 81861, 89265, 20686, 62420, 13001, 25890, 113485, 44069, 83808, 97501, 64722, 53047, 48275, 21145, 36240, 28863, 24720, 50385, 106352, 114373, 78852, 121282, 15056, 59508, 113244, 102623, 119619, 57729, 43483, 45155, 4184, 23061, 96908, 109342, 74276, 84797, 2689, 75451, 97699, 110922, 93719, 5, 20082, 76092, 101795, 25385, 79531, 96064, 40923, 87886, 4947, 19641, 38323, 46741, 16067, 34678, 21486, 94974, 67705, 26524, 77015, 120171, 82935, 62681, 119336, 118

# Run 2

In [7]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_od_all', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1752


In [8]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     175
jacc score:   18.2
norm jacc score: 0.104

PARSING ERROR
[110880, 11144]

NOT FULL SCORE
[45051, 94489, 92313]

ZERO SCORE
[42696, 11219, 38543, 117851, 98141, 62360, 28697, 18612, 6362, 11182, 62985, 26312, 97683, 63749, 29481, 81523, 87077, 71987, 5267, 110190, 60166, 3969, 45475, 11773, 62904, 107593, 102399, 6840, 46461, 118476, 49279, 90868, 48663, 82495, 70204, 105178, 87330, 11340, 90114, 54306, 33094, 16559, 113051, 63962, 4864, 86360, 2952, 55624, 74212, 94366, 104234, 41489, 16316, 73478, 7006, 92779, 61564, 28475, 83645, 46906, 50677, 107768, 26555, 3888, 55063, 68703, 98433, 102236, 82301, 12408, 121993, 40122, 48865, 16606, 120084, 40517, 108591, 33635, 108395, 86452, 6457, 37000, 26737, 28731, 81555, 16543, 16213, 25318, 77621, 45011, 17084, 49879, 11818, 25729, 120460, 17278, 60626, 109039, 81406, 6555, 95158, 14956, 117669, 85515, 52057, 52359, 66631, 40707, 64244, 69168, 103903, 79944, 105204, 1173, 11128, 56330, 2029, 88847, 80467, 120777, 114436, 365

# Run 3

In [9]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_od_all', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1751


In [10]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     177
jacc score:   24.666666666666664
norm jacc score: 0.13935969868173256

PARSING ERROR
[]

NOT FULL SCORE
[103234, 4797, 57218, 122032, 94793, 12883]

ZERO SCORE
[72010, 17424, 102774, 27056, 97636, 53403, 32606, 63981, 23909, 28739, 90633, 19890, 118198, 71983, 21148, 113145, 12737, 120447, 63853, 1456, 87200, 44795, 32579, 73159, 72190, 79282, 26776, 98529, 9808, 23407, 97719, 90152, 97405, 12822, 60959, 35954, 49133, 62301, 57852, 4965, 61739, 77487, 119983, 89830, 34666, 27539, 26986, 116362, 10425, 7558, 110204, 18387, 104375, 11033, 111469, 34250, 38490, 45212, 62378, 107372, 42250, 102052, 16146, 33418, 50524, 56006, 119257, 28253, 79329, 19873, 58452, 30878, 56644, 70931, 107282, 19395, 78595, 27783, 72540, 120918, 36166, 27617, 99218, 110401, 10227, 97785, 6478, 89683, 20340, 121668, 7407, 38381, 67113, 28094, 99065, 48743, 106168, 18042, 63049, 33216, 30292, 13720, 37852, 84903, 42612, 90782, 38718, 57255, 119698, 37018, 2264, 37825, 8799, 22192, 66214, 116839,

# Run 4

In [11]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_od_all', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1722


In [12]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     175
jacc score:   22.833333333333336
norm jacc score: 0.1304761904761905

PARSING ERROR
[5374, 64394]

NOT FULL SCORE
[28409, 101233, 34859, 115405]

ZERO SCORE
[19198, 114860, 84213, 61908, 105728, 37898, 94232, 88824, 39137, 120414, 42357, 54981, 97148, 67312, 100741, 73287, 2872, 108741, 46674, 112848, 56350, 48878, 85009, 60504, 100045, 58347, 112609, 13989, 42604, 98014, 119541, 111420, 69882, 115386, 41401, 81602, 104181, 49212, 91477, 14484, 120859, 67294, 112558, 623, 26833, 110723, 94646, 93525, 90459, 96813, 43717, 79516, 35102, 55286, 41594, 70674, 116403, 121017, 13796, 97737, 61319, 34059, 16591, 7398, 38580, 30067, 16523, 56324, 109817, 82810, 10823, 101636, 106338, 5000, 47714, 99219, 70136, 14594, 61614, 100757, 42286, 42003, 78783, 38768, 115726, 109566, 28343, 56471, 10674, 24484, 10881, 74665, 53395, 121644, 103248, 1120, 100561, 22928, 49397, 75625, 29251, 54038, 43129, 11713, 1112, 59317, 29320, 57365, 36078, 48468, 110919, 42795, 47096, 62756, 59123, 

# Run 5

In [13]:
train_index, test_index = next(splits)

train_set = [attempted_question[i] for i in train_index]
test_set = [attempted_question[i] for i in test_index]

total_examples = 0
with open('pos_eg_od_all', 'w') as out:
    for t in train_set:
        examples = parser.get_pos_example(t)
        total_examples += len(examples)
        for e in examples:
            print(e.gen_example(), file=out)
print(F'Total examples: {total_examples}')

Total examples: 1760


In [14]:
jacc, n_q = one_iter_gt(train_set, test_set)
total_jacc_score += jacc
total_questions += n_q
run_id += 1

# tests:     175
jacc score:   20.5
norm jacc score: 0.11714285714285715

PARSING ERROR
[56128, 116486]

NOT FULL SCORE
[82991, 5164, 90225]

ZERO SCORE
[69409, 44553, 16272, 40734, 69955, 54160, 50225, 113117, 4981, 101014, 43324, 33554, 580, 105758, 121269, 74869, 26504, 17298, 14773, 50297, 93301, 32889, 99745, 56096, 39904, 13292, 112862, 24971, 24144, 3796, 37031, 103778, 10649, 14277, 62242, 85532, 49716, 104027, 37284, 96969, 36414, 71850, 7962, 27275, 382, 14598, 48488, 9144, 95907, 78086, 120803, 96426, 77337, 18666, 79162, 31821, 60979, 72501, 112001, 87305, 94077, 42450, 101975, 67727, 74316, 74333, 28686, 19619, 27641, 65163, 677, 6753, 91784, 63786, 121553, 64344, 57942, 276, 53332, 25490, 40831, 60650, 58383, 17776, 35869, 42970, 64959, 40973, 84175, 77168, 51923, 119537, 93656, 1050, 78049, 64124, 41402, 51147, 66679, 101590, 14167, 15328, 25267, 101019, 45751, 87754, 84617, 114725, 40491, 106967, 93582, 32721, 91821, 39728, 101221, 81232, 26395, 45905, 3564, 80819, 5753

In [15]:
print(F'Total jacc score: {total_jacc_score}')
print(F'Total questions:  {total_questions}')
print(F'Avg norm jacc:    {total_jacc_score /  total_questions}')


Total jacc score: 112.03333333333333
Total questions:  879
Avg norm jacc:    0.12745544178991278


In [ ]:
from image_processing import draw_bounding_box
from PIL import Image
from IPython.display import display


test = a_q_dict[25720]

# time_span_to_timestamps_list(test)
vid_folder = frame_folder + test['vid_name'] + '/'

time = 30
qa_objects = od.get_frame_qa_objects(vid_folder, 0.7, time)
bboxes = [o.bbox for o in qa_objects]
display(Image.fromarray(draw_bounding_box(vid_folder + F'000{time}.jpg', bboxes)))